## Import the using package

In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import requests
import numpy as np
import sklearn
import monpa
import random
import pickle

08/25/2019 21:11:54 - INFO - monpa -   running on device cpu


+---------------------------------------------------------------------+
  Welcome to MONPA: Multi-Objective NER POS Annotator for Chinese
+---------------------------------------------------------------------+
Good, we can find the model file.


## The Architecture of the Designed Model using only Bi-GRU 
<img src="https://i.imgur.com/zjsZb0l.png" width = "300" height = "200" alt="design_model" align=center />

In [76]:
class Convolution_1D_Model (nn.Module):
    def __init__(self, total_tag, embeding_size, filter_num_1, filter_size_1, filter_num_2, filter_size_2 , output_size,max_word):
        super(Convolution_1D_Model, self).__init__()
        self.total_tag = total_tag
        self.max_word = max_word
        self.embeding_size = embeding_size
        self.filter_num_1 = filter_num_1
        self.filter_size_1 = filter_size_1
        self.filter_num_2 = filter_num_2
        self.filter_size_2 = filter_size_2
        self.output_size = output_size
        self.Embeding = nn.Embedding(self.total_tag, self.embeding_size)
        self.Conv1d_layer1 = nn.Sequential(nn.Conv1d(in_channels=self.embeding_size, 
                                                            out_channels=self.filter_num_1,
                                                            kernel_size=self.filter_size_1),
                                                         #nn.BatchNorm1d (self.filter_size_1),
                                                         nn.ReLU())
        
        self.Conv1d_layer2 = nn.Sequential(nn.Conv1d(in_channels=self.filter_num_1, 
                                                            out_channels=self.filter_num_2,
                                                            kernel_size=self.filter_size_2),
                                                         #nn.BatchNorm1d (self.filter_size_2),
                                                         nn.ReLU())
        self.Conv1d_layer3 = nn.Sequential(nn.Conv1d(in_channels=self.filter_num_2, 
                                                            out_channels=self.filter_num_2,
                                                            kernel_size=self.filter_size_2),
                                                         #nn.BatchNorm1d (self.filter_size_2),
                                                         nn.ReLU())
        self.Conv1d_layer4 = nn.Sequential(nn.Conv1d(in_channels=self.filter_num_2, 
                                                            out_channels=self.filter_num_2,
                                                            kernel_size=self.filter_size_2),
                                                         #nn.BatchNorm1d (self.filter_size_2)
                                                         nn.ReLU())
        self.Conv1d_layer5 = nn.Sequential(nn.Conv1d(in_channels=self.filter_num_2, 
                                                            out_channels=self.filter_num_2,
                                                            kernel_size=self.filter_size_2),
                                                         #nn.BatchNorm1d (self.filter_size_2),
                                                         nn.ReLU())
        self.Conv1d_layer6 = nn.Sequential(nn.Conv1d(in_channels=self.filter_num_2, 
                                                            out_channels=self.filter_num_2,
                                                            kernel_size=self.filter_size_2),
                                                         #nn.BatchNorm1d (self.filter_size_2),
                                                         nn.ReLU())
        
        # batch size is  set to 1 , output_size is set to 2
        L = self.max_word- self.filter_size_1+ 1
        for _ in range (4):
            L = L - self.filter_size_2 +1
        self.linear = nn.Linear( self.filter_num_2 * L, self.output_size )
        
    def forward(self, input_data):  
        embed_res = self.Embeding (input_data)
        if len(input_data) < self.max_word:
            for c in range (self.max_word - len(input_data)):
                embed_res = torch.cat((embed_res, torch.zeros(1, self.embeding_size)),dim = 0 )
        embed_res = embed_res.unsqueeze(0)
        #print (' Embedding :::: ' , embed_res.size())
        # let the input dimension be the (N , Cin , L) and then the output with the dimension (N, Cout , Lout)
        conv_res1 = self.Conv1d_layer1 (embed_res.permute(0,2,1))
        #print (' conv_res1 :::: ' , conv_res1.size())
        conv_res2 = self.Conv1d_layer2 (conv_res1)
        conv_res3 = self.Conv1d_layer3 (conv_res2)
        conv_res4 = self.Conv1d_layer4 (conv_res3)
        conv_res5 = self.Conv1d_layer5 (conv_res4)
        #print ('conv_res2 :::: ' , conv_res2.size())
        linear_res = self.linear(conv_res5.view(-1))
        final_res = F.log_softmax(linear_res)
        #print ("final_res :::: " , final_res)
        return final_res

In [77]:
model_v2 = Convolution_1D_Model(total_tag=16, embeding_size=8, filter_num_1=5, filter_size_1=2, filter_num_2=5, filter_size_2=2 ,output_size = 7, max_word=10)
test_vec = torch.LongTensor([1,2,3,4,5,6,7])
model_v2(test_vec)
print ('V2_Model_with_only_convolution1D ::: \n', model_v2)

V2_Model_with_only_convolution1D ::: 
 Convolution_1D_Model(
  (Embeding): Embedding(16, 8)
  (Conv1d_layer1): Sequential(
    (0): Conv1d(8, 5, kernel_size=(2,), stride=(1,))
    (1): ReLU()
  )
  (Conv1d_layer2): Sequential(
    (0): Conv1d(5, 5, kernel_size=(2,), stride=(1,))
    (1): ReLU()
  )
  (Conv1d_layer3): Sequential(
    (0): Conv1d(5, 5, kernel_size=(2,), stride=(1,))
    (1): ReLU()
  )
  (Conv1d_layer4): Sequential(
    (0): Conv1d(5, 5, kernel_size=(2,), stride=(1,))
    (1): ReLU()
  )
  (Conv1d_layer5): Sequential(
    (0): Conv1d(5, 5, kernel_size=(2,), stride=(1,))
    (1): ReLU()
  )
  (Conv1d_layer6): Sequential(
    (0): Conv1d(5, 5, kernel_size=(2,), stride=(1,))
    (1): ReLU()
  )
  (linear): Linear(in_features=25, out_features=7, bias=True)
)


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:67: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


## Data Pre-processing

* 將positive_data和negative_data載入存入List
* 將monpa的tags載入dictionary
* 將sentence 的 words 轉成詞性


In [78]:
'''training_data_sentence_list = list()
training_data_tag_list = list()
training_target_sentence_list = list()
dict_idx2tag = dict()
dict_tag2idx = dict()

# Read both the postive data and negative data
with open ('../CJX_Train_Test_Data/positive_data.txt', 'r', encoding = 'utf-8') as rf:
    for pos_s in rf.readlines():
        training_data_sentence_list.append(pos_s.strip('\n').strip(',').strip('，').strip('《').strip('》').strip('【').strip('】').strip('、').strip('、').strip('-'))
        training_target_sentence_list.append(1)
with open ('../CJX_Train_Test_Data/positive_data.txt', 'r', encoding = 'utf-8') as rf:
    for pos_s in rf.readlines():
        training_data_sentence_list.append(pos_s.strip('\n').strip(',').strip('，').strip('《').strip('》').strip('【').strip('】').strip('、').strip('-'))
        training_target_sentence_list.append(0)
        
print ('training data size : ',len(training_data_sentence_list))
print ('training data target size : ',len(training_target_sentence_list))
tmp_cnt = 0
# Change the words to tags, because we need to use the sequential tags for training.
for sentence in training_data_sentence_list:
    tmp_cnt += 1
    #print (tmp_cnt)
    tmp = monpa.pseg(sentence)
    tmp_list = list()
    for item in tmp:
        tmp_list.append(item[1])
    training_data_tag_list.append(tmp_list)
    
print (len(training_data_tag_list))

# Read the tags of the monpa
with open ('./monpa_tag.txt' , 'r' , encoding = 'utf-8') as rf:
    cnt = 0
    for tag in rf.readlines():
        dict_idx2tag[cnt] = tag.strip('\n')
        dict_tag2idx[tag.strip('\n')] = cnt
        cnt += 1  

print ('The dictionary of idxs 2 tags : ', dict_idx2tag)
print ('The dictionary of tags 2 idxs : ' , dict_tag2idx)

shuffle_zip = list(zip(training_data_tag_list, training_target_sentence_list))
random.shuffle(shuffle_zip)
training_data_tag_list[:], training_target_sentence_list[:] = zip(*shuffle_zip)
print (training_data_tag_list, training_target_sentence_list)
'''


"training_data_sentence_list = list()\ntraining_data_tag_list = list()\ntraining_target_sentence_list = list()\ndict_idx2tag = dict()\ndict_tag2idx = dict()\n\n# Read both the postive data and negative data\nwith open ('../CJX_Train_Test_Data/positive_data.txt', 'r', encoding = 'utf-8') as rf:\n    for pos_s in rf.readlines():\n        training_data_sentence_list.append(pos_s.strip('\n').strip(',').strip('，').strip('《').strip('》').strip('【').strip('】').strip('、').strip('、').strip('-'))\n        training_target_sentence_list.append(1)\nwith open ('../CJX_Train_Test_Data/positive_data.txt', 'r', encoding = 'utf-8') as rf:\n    for pos_s in rf.readlines():\n        training_data_sentence_list.append(pos_s.strip('\n').strip(',').strip('，').strip('《').strip('》').strip('【').strip('】').strip('、').strip('-'))\n        training_target_sentence_list.append(0)\n        \nprint ('training data size : ',len(training_data_sentence_list))\nprint ('training data target size : ',len(training_target_sen

## Save the training data list

In [79]:
'''f = open ('../pickle/training_tags.pkl', 'wb')
pickle.dump(training_data_tag_list, f)
f.close()
f = open ('../pickle/target.pkl', 'wb')
pickle.dump(training_target_sentence_list, f)
f.close()'''

"f = open ('../pickle/training_tags.pkl', 'wb')\npickle.dump(training_data_tag_list, f)\nf.close()\nf = open ('../pickle/target.pkl', 'wb')\npickle.dump(training_target_sentence_list, f)\nf.close()"

In [80]:
f = open ('../pickle/training_tags.pkl', 'rb')
a = pickle.load(f)
f.close()
f = open ('../pickle/target.pkl', 'rb')
b = pickle.load(f)
f.close()

In [81]:
dict_idx2tag = dict()
dict_tag2idx = dict()
with open ('./monpa_tag.txt' , 'r' , encoding = 'utf-8') as rf:
    cnt = 0
    for tag in rf.readlines():
        dict_idx2tag[cnt] = tag.strip('\n')
        dict_tag2idx[tag.strip('\n')] = cnt
        cnt += 1  
training_data_tag_list = a
training_target_sentence_list = b

## Hyperparamters_v2

In [85]:
epoch_v2 = 1000
learning_rate_v2 = 0.00001
input_size_v2 = len(dict_idx2tag.keys())
gru_hidden_size_v2 = 8
embedding_size_v2 = 8
output_size_v2 = 2
gru_layer_v2 = 1
filter_num_1_v2 = 3
filter_size_1_v2 = 2
filter_num_2_v2 = 3
filter_size_2_v2 = 2
max_word_v2 = 20

## 定義訓練function

In [86]:
def train_v2():
    model_v2 = Convolution_1D_Model(total_tag= input_size_v2, 
                                    embeding_size= embedding_size_v2, 
                                    filter_num_1= filter_num_1_v2, 
                                    filter_size_1= filter_size_1_v2, 
                                    filter_num_2= filter_num_2_v2, 
                                    filter_size_2=filter_size_2_v2 ,
                                    output_size = output_size_v2, 
                                    max_word= max_word_v2 )
    optimizer_v2 = torch.optim.SGD(model_v2.parameters(), lr=learning_rate_v2)
    loss_function_v2 = nn.NLLLoss()
    
    for epoch in range(epoch_v2):
        sub_epoch = 0
        total_loss = 0
        for seq_tag_idx in range(len(training_data_tag_list)):
            if len(training_data_tag_list[seq_tag_idx]) > max_word_v2:
                continue
            sub_epoch += 1
            input_data = torch.LongTensor([dict_tag2idx[t] for t in training_data_tag_list[seq_tag_idx]])
            target_data = torch.LongTensor([training_target_sentence_list[seq_tag_idx]])
            out = model_v2(input_data)
            out = out.unsqueeze(0)
            loss = loss_function_v2(out, target_data)
            total_loss += loss.item()
            optimizer_v2.zero_grad()
            loss.backward()
            optimizer_v2.step()
            '''
            if sub_epoch % 100 == 0:
                print ('Epoch '+  str(epoch) + ' '+ str(sub_epoch)+'/' + str(len(training_data_tag_list)) + '  Loss : ' + str(total_loss))
            '''
        print ('Epoch ' + str(epoch) + ' Loss : ' + str (total_loss) + ' total_train_data : ' + str(sub_epoch) )
        torch.save(model_v2, '../pickle/model_v2.pt')

In [87]:
train_v2()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:67: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 0 Loss : 4861.99290817976 total_train_data : 7014


/anaconda3/lib/python3.6/site-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Convolution_1D_Model. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch 1 Loss : 4861.977296233177 total_train_data : 7014
Epoch 2 Loss : 4861.962986171246 total_train_data : 7014
Epoch 3 Loss : 4861.949198305607 total_train_data : 7014
Epoch 4 Loss : 4861.935727536678 total_train_data : 7014
Epoch 5 Loss : 4861.922657608986 total_train_data : 7014
Epoch 6 Loss : 4861.910870611668 total_train_data : 7014
Epoch 7 Loss : 4861.90072286129 total_train_data : 7014
Epoch 8 Loss : 4861.891461312771 total_train_data : 7014
Epoch 9 Loss : 4861.882511198521 total_train_data : 7014
Epoch 10 Loss : 4861.8738723397255 total_train_data : 7014
Epoch 11 Loss : 4861.86531996727 total_train_data : 7014
Epoch 12 Loss : 4861.85710310936 total_train_data : 7014
Epoch 13 Loss : 4861.849670886993 total_train_data : 7014
Epoch 14 Loss : 4861.843432128429 total_train_data : 7014
Epoch 15 Loss : 4861.837845504284 total_train_data : 7014
Epoch 16 Loss : 4861.832608640194 total_train_data : 7014
Epoch 17 Loss : 4861.8275300860405 total_train_data : 7014
Epoch 18 Loss : 4861.822

Epoch 142 Loss : 4861.752418458462 total_train_data : 7014
Epoch 143 Loss : 4861.7524127960205 total_train_data : 7014
Epoch 144 Loss : 4861.752415299416 total_train_data : 7014
Epoch 145 Loss : 4861.752420127392 total_train_data : 7014
Epoch 146 Loss : 4861.752434015274 total_train_data : 7014
Epoch 147 Loss : 4861.752408087254 total_train_data : 7014
Epoch 148 Loss : 4861.752416729927 total_train_data : 7014
Epoch 149 Loss : 4861.752417743206 total_train_data : 7014
Epoch 150 Loss : 4861.752416908741 total_train_data : 7014
Epoch 151 Loss : 4861.75243639946 total_train_data : 7014
Epoch 152 Loss : 4861.752413213253 total_train_data : 7014
Epoch 153 Loss : 4861.752420127392 total_train_data : 7014
Epoch 154 Loss : 4861.752407610416 total_train_data : 7014
Epoch 155 Loss : 4861.752406656742 total_train_data : 7014
Epoch 156 Loss : 4861.752424061298 total_train_data : 7014
Epoch 157 Loss : 4861.752394735813 total_train_data : 7014
Epoch 158 Loss : 4861.752418100834 total_train_data : 70

...

Epoch 420 Loss : 4861.752335131168 total_train_data : 7014
Epoch 421 Loss : 4861.752363562584 total_train_data : 7014
Epoch 422 Loss : 4861.75235170126 total_train_data : 7014
Epoch 423 Loss : 4861.7523357868195 total_train_data : 7014
Epoch 424 Loss : 4861.75232988596 total_train_data : 7014
Epoch 425 Loss : 4861.752321124077 total_train_data : 7014
Epoch 426 Loss : 4861.752361595631 total_train_data : 7014
Epoch 427 Loss : 4861.752358019352 total_train_data : 7014
Epoch 428 Loss : 4861.752336680889 total_train_data : 7014
Epoch 429 Loss : 4861.752335071564 total_train_data : 7014
Epoch 430 Loss : 4861.752328038216 total_train_data : 7014
Epoch 431 Loss : 4861.752347290516 total_train_data : 7014
Epoch 432 Loss : 4861.752367019653 total_train_data : 7014
Epoch 433 Loss : 4861.75235003233 total_train_data : 7014
Epoch 434 Loss : 4861.7523428201675 total_train_data : 7014
Epoch 435 Loss : 4861.752321600914 total_train_data : 7014
Epoch 436 Loss : 4861.752336978912 total_train_data : 701

Epoch 560 Loss : 4861.752357125282 total_train_data : 7014
Epoch 561 Loss : 4861.752325832844 total_train_data : 7014
Epoch 562 Loss : 4861.752323389053 total_train_data : 7014
Epoch 563 Loss : 4861.752316534519 total_train_data : 7014
Epoch 564 Loss : 4861.752334356308 total_train_data : 7014
Epoch 565 Loss : 4861.752352535725 total_train_data : 7014
Epoch 566 Loss : 4861.752336919308 total_train_data : 7014
Epoch 567 Loss : 4861.752335011959 total_train_data : 7014
Epoch 568 Loss : 4861.752326846123 total_train_data : 7014
Epoch 569 Loss : 4861.752315342426 total_train_data : 7014
Epoch 570 Loss : 4861.752349555492 total_train_data : 7014
Epoch 571 Loss : 4861.752336859703 total_train_data : 7014
Epoch 572 Loss : 4861.75232899189 total_train_data : 7014
Epoch 573 Loss : 4861.752314150333 total_train_data : 7014
Epoch 574 Loss : 4861.752304136753 total_train_data : 7014
Epoch 575 Loss : 4861.752344787121 total_train_data : 7014
Epoch 576 Loss : 4861.752354443073 total_train_data : 701

Epoch 699 Loss : 4861.752328634262 total_train_data : 7014
Epoch 700 Loss : 4861.752325892448 total_train_data : 7014
Epoch 701 Loss : 4861.752341926098 total_train_data : 7014
Epoch 702 Loss : 4861.752346634865 total_train_data : 7014
Epoch 703 Loss : 4861.752331674099 total_train_data : 7014
Epoch 704 Loss : 4861.752342045307 total_train_data : 7014
Epoch 705 Loss : 4861.752350866795 total_train_data : 7014
Epoch 706 Loss : 4861.752351522446 total_train_data : 7014
Epoch 707 Loss : 4861.752342641354 total_train_data : 7014
Epoch 708 Loss : 4861.752338826656 total_train_data : 7014
Epoch 709 Loss : 4861.752320587635 total_train_data : 7014
Epoch 710 Loss : 4861.752314984798 total_train_data : 7014
Epoch 711 Loss : 4861.752325594425 total_train_data : 7014
Epoch 712 Loss : 4861.752321004868 total_train_data : 7014
Epoch 713 Loss : 4861.75231641531 total_train_data : 7014
Epoch 714 Loss : 4861.752327084541 total_train_data : 7014
Epoch 715 Loss : 4861.752328634262 total_train_data : 701

Epoch 839 Loss : 4861.75231218338 total_train_data : 7014
Epoch 840 Loss : 4861.75231730938 total_train_data : 7014
Epoch 841 Loss : 4861.75231385231 total_train_data : 7014
Epoch 842 Loss : 4861.752324581146 total_train_data : 7014
Epoch 843 Loss : 4861.752328634262 total_train_data : 7014
Epoch 844 Loss : 4861.752325773239 total_train_data : 7014
Epoch 845 Loss : 4861.752331614494 total_train_data : 7014
Epoch 846 Loss : 4861.752336144447 total_train_data : 7014
Epoch 847 Loss : 4861.752331614494 total_train_data : 7014
Epoch 848 Loss : 4861.752342045307 total_train_data : 7014
Epoch 849 Loss : 4861.752346992493 total_train_data : 7014
Epoch 850 Loss : 4861.752346813679 total_train_data : 7014
Epoch 851 Loss : 4861.752325594425 total_train_data : 7014
Epoch 852 Loss : 4861.752334415913 total_train_data : 7014
Epoch 853 Loss : 4861.752320587635 total_train_data : 7014
Epoch 854 Loss : 4861.752312839031 total_train_data : 7014
Epoch 855 Loss : 4861.75231218338 total_train_data : 7014
E

Epoch 979 Loss : 4861.752326250076 total_train_data : 7014
Epoch 980 Loss : 4861.752314925194 total_train_data : 7014
Epoch 981 Loss : 4861.752314269543 total_train_data : 7014
Epoch 982 Loss : 4861.752316296101 total_train_data : 7014
Epoch 983 Loss : 4861.752304553986 total_train_data : 7014
Epoch 984 Loss : 4861.752301752567 total_train_data : 7014
Epoch 985 Loss : 4861.752315104008 total_train_data : 7014
Epoch 986 Loss : 4861.752318024635 total_train_data : 7014
Epoch 987 Loss : 4861.752321064472 total_train_data : 7014
Epoch 988 Loss : 4861.7523247003555 total_train_data : 7014
Epoch 989 Loss : 4861.752333879471 total_train_data : 7014
Epoch 990 Loss : 4861.752324402332 total_train_data : 7014
Epoch 991 Loss : 4861.752324581146 total_train_data : 7014
Epoch 992 Loss : 4861.752335906029 total_train_data : 7014
Epoch 993 Loss : 4861.752348005772 total_train_data : 7014
Epoch 994 Loss : 4861.75233733654 total_train_data : 7014
Epoch 995 Loss : 4861.752326250076 total_train_data : 70

In [88]:
def predict():
    model_v2 = Convolution_1D_Model(total_tag= input_size_v2, 
                                    embeding_size= embedding_size_v2, 
                                    filter_num_1= filter_num_1_v2, 
                                    filter_size_1= filter_size_1_v2, 
                                    filter_num_2= filter_num_2_v2, 
                                    filter_size_2=filter_size_2_v2 ,
                                    output_size = output_size_v2, 
                                    max_word= max_word_v2 )
    model_v2 = torch.load( '../pickle/model_v2.pt')
    total_num = 0
    correct = 0
    for seq_tag_idx in range(len(training_data_tag_list)):
        
        if len(training_data_tag_list[seq_tag_idx]) > max_word_v2:
            continue
        total_num += 1
        input_data = torch.LongTensor([dict_tag2idx[t] for t in training_data_tag_list[seq_tag_idx]])
        target_data = torch.LongTensor([training_target_sentence_list[seq_tag_idx]])
        out = model_v2(input_data)
        
        if out[0].item() > out[1].item() and target_data[0].item() ==0:
            correct += 1
        elif out[0].item() < out[1].item() and target_data[0].item() ==1:
            correct += 1
        else:
            continue
    print (correct / total_num )
        

In [89]:
predict()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:67: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


0.5
